## 결정 트리, 랜덤 포레스트, 에이다부스트. 
#### 필요한 패키지를 불러온다:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
from scipy.stats import itemfreq

#### 데이터를 불러온다:

In [2]:
os.chdir(r"C:\Users\bit\Downloads\머신러닝 알고리즘과 응용\data")

In [3]:
# df = pd.read_csv('data_spam.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_spam.csv', header='infer',encoding='latin1')

In [4]:
X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns

'yes', 'no' 레이블을 숫자로 변환:

In [19]:
LE = preprocessing.LabelEncoder() #독립변수가 아닌 종속 변수(라벨)에 대해 사용한다.
Y = LE.fit_transform(Y)

In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()  #문자열이나 정수로된 라벨 값을  0 ~  K−1 까지의 정수로 변환
y = ['A', 'B', 'A', 'A', 'B', 'C', 'C', 'A', 'C', 'B', 'D', 'd']
le.fit(y)
le.classes_     #변환된 규칙은 classes_ 속성에서 확인

array(['A', 'B', 'C', 'D', 'd'], dtype='<U1')

In [26]:
y2 = le.transform(y) #변환
y2

array([0, 1, 0, 0, 1, 2, 2, 0, 2, 1, 3, 4], dtype=int64)

In [ ]:
le.inverse_transform(y2)   #역변환
#예측 결과에 적용할 수 있도록 역변환을 위한 inverse_transform 메서드도 지원

In [6]:
table = itemfreq(Y)
table

C:\Users\bit\PycharmProjects\python\venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[  0, 736],
       [  1, 264]], dtype=int64)

NaN이 있으면 채워 넣음:

In [14]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

#### 데이터 전처리:

In [8]:
X = preprocessing.scale(X)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)

#### Tree 적용 (교차검증 최적화 포함):

In [34]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}
parameters

{'max_depth': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30])}

In [36]:
gridCV = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']
best_depth

6

In [12]:
print("Tree best depth : " + str(best_depth))

Tree best depth : 18


In [13]:
DTC_best = DecisionTreeClassifier(max_depth=best_depth)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Tree best accuracy : 0.88


#### Random Forest 적용 (교차검증 최적화 포함):

Tree 하고는 스텝에서 조금 차이가 있음:

In [38]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
#GridSearchCV = 그리드를 사용한 복수 하이퍼 파라미터 최적화
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']

In [39]:
estimator_grid = np.arange(1, 30, 2)
estimator_grid

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29])

In [30]:
depth_grid = np.arange(1, 10, 2)
depth_grid

array([1, 3, 5, 7, 9])

In [40]:
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
parameters

{'max_depth': array([1, 3, 5, 7, 9]),
 'n_estimators': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29])}

In [41]:
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
#cv :  모든 교차 검증결과

In [42]:
gridCV.fit(X_train, Y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]), 'max_depth': array([1, 3, 5, 7, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [37]:
print("Random Forest best n estimator : " + str(best_n_estim))
print("Random Forest best depth : " + str(best_depth))

Random Forest best n estimator : 21
Random Forest best depth : 6


In [17]:
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(X_train, Y_train);
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Random Forest best accuracy : 0.932


#### AdaBoost 적용 (교차검증 최적화 포함): <br>
디폴트: base_estimator=DecisionTreeClassifier 

In [ ]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']

In [ ]:
print("Ada Boost best n estimator : " + str(best_n_estim))
print("Ada Boost best learning rate : " + str(best_learn_rate))

In [ ]:
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(X_train, Y_train);
Y_pred = AB_best.predict(X_test)
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))